# Procesamiento con pipelines

In [5]:
import pandas as pd
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
# Importamos la librería que necesitamos
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score # métrica de evaluación
from sklearn.metrics import classification_report
from sklearn import metrics
import sys
sys.path.append('../Code')
from CustomTransformers import CollinearityDropper, ColumnSelector, ColumnScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


## Preprocesamiento usando transformadores "customizados"

In [6]:
clfs = []
df = pd.read_csv('https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/asteroids_nasa.csv')
X = df.drop("Hazardous",axis=1)
y = df["Hazardous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y, random_state=0)


In [7]:
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
pl = Pipeline(steps=[("QuitarColineales", CollinearityDropper(columns=None, min_coef=0.9)),
                     ("SeleccionoColumnas", ColumnSelector(columns=['Absolute Magnitude', 'Minimum Orbit Intersection'])),
                     #Si llamo al estandar escaler pierdo las columnas, esta mal? Para nada, pero quiero conservarlas
                     ("Escalado", StandardScaler()),
                    #("Escalado", ColumnScaler(scaler=StandardScaler())), #Si columnas=None, se escalan todas    
                     ("Clasificador", GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, scoring='accuracy', cv=5))
                     ])

In [9]:
#
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.992181947405828


## Hacemos lo mismo con column transformers